# အဆင့်လိုက် အဖွဲ့ဝင် Middleware ဖြင့် ဟိုတယ်ဦးချုပ်ခြင်း

ဤ notebook သည် Microsoft Agent Framework ကို အသုံးပြုသော **function-based middleware** ကို ဖော်ပြသည်။ conditional workflow နမူနာပေါ်တွင် ထပ်ကျပ်ပြီး အဆင့်လိုက် အဖွဲ့ဝင်များအား အထူးအခွင့်အရေးများ ပေးသော middleware အလွှာတစ်ခု ထည့်သွင်းသည်။

## သင်ယူမည့်အရာများ:
1. **Function-Based Middleware**: function ရလဒ်များအား တားဆီး ပြင်ဆင်ခြင်း
2. **Context Access**: အကောင်အထည်ဖော်ပြီးနောက် `context.result` ကို ဖတ်ရှု ပြင်ဆင်ခြင်း
3. **စီးပွားရေး လုပ်ငန်း Logic အကောင်အထည်ဖော်ခြင်း**: အဆင့်လိုက် အဖွဲ့ဝင် အကျိုးခံစားခွင့်များ
4. **ရလဒ်ပြောင်းလဲခြင်း**: အသုံးပြုသူ အခြေအနေအရ function ရလဒ်များ ပြောင်းလဲခြင်း
5. **တူညီ Workflow၊ မတူညီသောရလဒ်များ**: Middleware အားကောင်းမှ ထိန်းချုပ်ခြင်းဖြင့် ပြုလုပ်နေသော အပြုအမူများ

## Middleware ပါသော Workflow ပုံစံ:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Conditional Workflow နှင့် ကွာခြားချက် အရေးပါတဲ့:

**Middleware မပါသော အခြေအနေ** (14-conditional-workflow.ipynb):
- ပားရစ်တွင် အခန်းမရှိ → alternative_agentသို့ လမ်းညွှန်ခြင်း

**Middleware ပါသော အခြေအနေ** (ဤ notebook):
- ပုံမှန် အသုံးပြုသူ + ပားရစ် → အခန်းမရှိ → alternative_agentသို့ လမ်းညွှန်ခြင်း
- အဆင့်လိုက် အသုံးပြုသူ + ပားရစ် → 🌟 Middleware မှ override → ရနိုင် → booking_agent သို့ လမ်းညွှန်ခြင်း

## လိုအပ်ချက်များ:
- Microsoft Agent Framework ထည့်သွင်းထားရှိရမည်
- Conditional workflows အတွက် နားလည်မှုရှိရမည် (14-conditional-workflow.ipynb ကို ကြည့်ရှုပါ)
- GitHub token သို့မဟုတ် OpenAI API key
- Middleware စနစ်များ အကြမ်းဖျဥ်းနားလည်မှု


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## အဆင့် ၁: ဖွဲ့စည်းထားသော ထွက်ရလဒ်များအတွက် Pydantic မော်ဒယ်များ သတ်မှတ်ခြင်း

ဤမော်ဒယ်များသည် သေချာသော **schema** ကို ကိုယ်စားပြုသည်၊ ၎င်းကိုယ်စား (agents) အနေဖြင့် ပြန်လည်ပို့ဆောင်မည့် ဖောင်များဖြစ်သည်။ Middleware မှ အဆင့်တန်ဖိုး (availability result) ကို ပြင်ဆင်သည့်အခါ တွက်ချက်ရန် `priority_override` ရက်ပုံမှတ်ကို ထည့်သွင်းထားပါသည်။


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: အရေးကြီး အဖွဲ့ဝင်များ ဒေတာဘေ့စ် ကို သတ်မှတ်ခြင်း

ဒီ ဒီမိုအတွက်တော့ အရေးကြီး အဖွဲ့ဝင် ဒေတာဘေ့စ်ကို စာနည်းပြမည်ဖြစ်သည်။ ထုတ်လုပ်မှုတွင်တော့ အမှန်တကယ် ဒေတာဘေ့စ် သို့မဟုတ် API ကို မေးမြန်းမည် ဖြစ်သည်။

**အရေးကြီး အဖွဲ့ဝင်များ:**
- `alice@example.com` - VIP အဖွဲ့ဝင်
- `bob@example.com` - Premium အဖွဲ့ဝင်  
- `priority_user` - စမ်းသပ်အကောင့်


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## အဆင့် ၃: ဟိုတယ်မှာကြိုတင်ဘွတ်ကင်ကိရိယာကို ဖန်တီးပါ

ကန့်သတ်ချက်ရှိတဲ့ စနစ်လည်ပတ်မှုနဲ့ တူပြီး၊ ယခုမှာတော့ middleware မှတဆင့် လုပ်ဆောင်ချက်ကို ရပ်တန့်ပြီး ကြားဖြတ်မယ်!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Step 4: 🌟 ဦးစားပေးစစ်ဆေးမှု Middleware ဖန်တီးခြင်း (အဓိက အင်္ဂါရပ်!)

ဤသည်မှာ ဤ notebook ၏ **အခြေခံ လုပ်ဆောင်ချက်** ဖြစ်သည်။ Middleware သည် -

1. hotel_booking ဖန်တီးမှုခေါ်ဆိုမှုကို **တားဆီးသည်**
2. `next(context)` ကိုခေါ်၍ လုပ်ဆောင်ချက်ကို သာမာန်အတိုင်း **ပြုလုပ်သည်**
3. `context.result` တွင်ရလဒ်ကို **စစ်ဆေးသည်**
4. အသုံးပြုသူ ဦးစားပေးဖြစ်ပြီး အခန်း မရှိပါက ရလဒ်အား **အစားထိုးသည်**
5. ပြောင်းလဲပြီးသော ရလဒ်ကို မန်နေဂျာထံ **ပြန်ပို့သည်**

**အဓိက ပုံစံ:**
```python
async def my_middleware(context, next):
    await next(context)  # လုပ်ဆောင်ချက်ကို အကောင်အထည်ဖော်ပါ
    # ယခု context.result မှာ လုပ်ဆောင်ချက်ရလဒ် ပါဝင်သည်
    if some_condition:
        context.result = new_value  # မူရင်းကို အစားထိုးပါ!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Step 5: Routing အတွက် အခြေအနေ Function များကို ဆိုက်စစ်ခြင်း

Conditional workflow တူညီသော အခြေအနေ function များ - ၎င်းတို့သည် ရှုမြင်သည့် output ကို စစ်ဆေးပြီး routing ကို ဆုံးဖြတ်သည်။


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## အဆင့် ၆: စိတ်ကြိုက် ပြသမှု အကောင်အထည်ဖော်သူ ဖန်တီးခြင်း

မတိုင်မီရှိခဲ့သည့် အကောင်အထည်ဖော်သူတူညီပြီး - နောက်ဆုံး workflow ထွက်ရှိမှုကို ပြသသည်။


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## အဆင့် ၇: ပတ်ဝန်းကျင်အနေအထားများကို လွှင့်ယူပါ

LLM client (GitHub Models သို့ OpenAI) ကို စနစ်တကျ ပြင်ဆင်ပါ။


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Step 8: Middleware ဖြင့် AI Agent များ ဖန်တီးခြင်း

**အဓိကကွာခြားချက်:** availability_agent ကို ဖန်တီးရာတွင် `middleware` ပါရာမီတာကို ပေးပို့ပါသည်!

ဒါကနေ priority_check_middleware ကို agent ရဲ့ function invocation လမ်းကြောင်းထဲသို့ ထည့်သွင်းတင်သွင်းတာဖြစ်ပါတယ်။


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Step 9: Workflow ကို တည်ဆောက်ခြင်း

ယခင်ကဲ့သို့တူညီသော workflow ဖွဲ့စည်းမှု - ရနိုင်မှု အပေါ်မှာ အခြေခံပြီး စည်းကြပ်လိုက်ခြားခြင်း။


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 10: စမ်းသပ်မှုအမှု ၁ - ပဲရစ်ရှိ ပုံမှန်အသုံးပြုသူ (မအတိုးအကျယ်ပြုလုပ်ခြင်း)

ပုံမှန်အသုံးပြုသူတစ်ဦးသည် ပဲရစ်ကို အခန်းဘွတ်ကင်လုပ်ရန်ကြိုးစားသည် → အခန်းမရှိ → alternative_agent သို့ ပို့ဆောင်မှုကြောင်းသွားသည်


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Step 11: Test Case 2 - 🌟 Priority User in Paris (WITH Override!)

A priority member tries to book Paris → No rooms initially → 🌟 Middleware overrides! → Routes to booking_agent

**ဤသည်မှာ middleware အင်အား၏ အဓိက ပြသချက်ဖြစ်ပါသည်!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## အဆင့် ၁၂: စမ်းသပ်မှုအမှု ၃ - စတော့ခိုးလ်မ်ရှိ ဦးစားပေးအသုံးပြုသူ (အထူးရရှိနိုင်ပြီးသား)

ဦးစားပေးအသုံးပြုသူသည် စတော့ခိုးလ်မ်ကိုကြိုးစားသည် → အခန်းများရနိုင်သည် → override လိုအပ်မှုမရှိ → booking_agent သို့လမ်းကြောင်းပြောင်းသည်

ဒါက middleware သည် လိုအပ်သောအချိန်၌သာ အက်ရှင်လုပ်သည်ဆိုတာကို ဖော်ပြနေသည်!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## အဓိကယူဆချက်များနှင့် Middleware အကြောင်းအရာများ

### ✅ သင်သိခဲ့သည်များ-

#### **1. Function-Based Middleware ပုံစံ**

Middleware သည် function အခေါ်များအား စာနယ်ဇင်း async function တစ်ခု အသုံးပြု၍ ထိတွေ့စွဲဆောင်သည်-

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # ဖန်ရှင်ကို အလုပ်လုပ်ခွင့်မပြုမှီ
    print("Intercepting...")
    
    # ဖန်ရှင်ကို အလုပ်လုပ်ပါ
    await next(context)
    
    # ဖန်ရှင် အလုပ်ပြီးနောက် - ရလဒ်ကို စစ်ဆေးပါ
    if context.result:
        # လိုအပ်ပါက ရလဒ်ကို ပြင်ဆင်ပါ
        context.result = modified_value
```

#### **2. Context တစ်ခု လက်လှမ်းထိခြင်းနှင့် ရလဒ်အစားထိုးခြင်း**

- `context.function` - ခေါ်ဆိုနေသော function ကို လက်လှမ်းထိရန်
- `context.arguments` - function argument များဖတ်ရန်
- `context.kwargs` - အပို parameter များ လက်လှမ်းထိရန်
- `await next(context)` - function ကို ဆောင်ရွက်ရန်
- `context.result` - function ၏ output ကို ဖတ်/ပြင်ဆင်ရန်

#### **3. စီးပွားရေး လုပ်ငန်းဖြေရှင်းချက် အကောင်အထည်ဖော်ခြင်း**

ကျွန်တော်တို့ middleware သည် ဦးစားပေး အဖွဲ့ဝင် အကျိုးခံစားခွင့်များကို ဆောင်ရွက်သည်-
- **ပုံမှန် အသုံးပြုသူများ**: မပြောင်းလဲမှု၊ စံချိန် workflow
- **ဦးစားပေး အသုံးပြုသူများ**: "မရရှိနိုင်ခြင်း" ကို "ရရှိနိုင်" သို့ အစားထိုး
- **အခြေအနေ logic**: လိုအပ်သည့်အချိန်တွင်သာ အစားထိုးသည်

#### **4. workflow တူညီသေး၍ ရလဒ်ကွဲပြားခြင်း**

Middleware ၏ အင်အား-
- ✅ workflow ပုံစံမပြောင်းလဲပါ
- ✅ tool function မပြောင်းလဲပါ
- ✅ conditional routing logic မပြောင်းလဲပါ
- ✅ middlewareသာ → မတူညီသောမူဝါဒ!

### 🚀 အကောင်အထည်ဖော်နိုင်သောကျုံ့ကြံမှုများ-

1. **VIP/Premium Feature များ**
   - Premium အသုံးပြုသူများအတွက် rate limit ကို အစားထိုးခွင့်ပြုသည်
   - အရင်းအမြစ်များကို ဦးစားပေး လက်လှမ်းမီခွင့်ပေးသည်
   - Premium feature များကို dynamic unlock လုပ်ခြင်း

2. **A/B စမ်းသပ်မှုများ**
   - အသုံးပြုသူများကို မတူညီသော ဖောင်များသို့ လမ်းကြောင်းပေးခြင်း
   - အသုံးပြုသူ အချို့ ဖြင့် အသစ် feature များစမ်းသပ်ခြင်း
   - Gradual feature rollout များ

3. **လုံခြုံရေးနှင့် နှိုင်းယှဉ်မှု**
   - function ခေါ်ဆိုမှုများ audit လုပ်ခြင်း
   - သိတတ်ရပ်တန့်ခြင်း လုပ်ငန်းများတားမြစ်ခြင်း
   - စီးပွားရေး စည်းကမ်းများ တင်းကြပ်စွာထားခြင်း

4. **လုပ်ဆောင်မှုတိုးတက်မှု**
   - အထူး user များအတွက် 결과 များ cache ထားခြင်း
   - စျေးကြီးသော အလုပ်များ ကျော်လွှားခြင်း
   - ရင်းမြစ်များကို dynamic ခွဲဝေဖြန့်ဖြူးခြင်း

5. **အမှားကောင်းချယူခြင်းနှင့် ပြန်လည်ကြိုးစားခြင်း**
   - အမှားများ catch လုပ်ပြီး တမန်ဆောင်ရွက်ခြင်း
   - ပြန်ကြိုးစားမှု logic ဆောင်ရွက်ခြင်း
   - အခြား ဖြေရှင်းမှုများသို့ fallback ပြုလုပ်ခြင်း

6. **မှတ်တမ်းတင်ခြင်းနှင့် မော်နီတာလုပ်ခြင်း**
   - function အချိန်နှင့် သက်ဆိုင်သည့် စီမံခန့်ခွဲမှုများ လုပ်ခြင်း
   - parameter နှင့် ရလဒ် များ log ထားခြင်း
   - အသုံးပြုမှု ပုံစံများ မော်နီတာခြင်း

### 🔑 Decorator များနှင့် ယှဉ်ပြိုင်မှု အချက်များ:

| လက္ခဏာ | Decorator | Middleware |
|---------|-----------|------------|
| **Scope** | တစ်ခုတည်း function | agent တွင်ရှိသော function များအားလုံး |
| **Flexibility** | သတ်မှတ်ချက်တွင် သေချာထား | runtime တွင် dynamic |
| **Context** | ကန့်သတ်ထား | agent context အပြည့်အဝ |
| **Composition** | decorator များစုံ | middleware pipeline |
| **Agent-Aware** | မရှိ | ရှိ (agent state ကို လက်လှမ်းထိ) |

### 📚 Middleware ကို ဘယ်အချိန်သုံးသင့်သလဲ-

✅ **middleware သုံးသင့်သော အချိန်များ-**
- အသုံးပြုသူ/ session အခြေအနေအပေါ် မူတည်၍ အပြုအမူပြောင်းလိုချင်သည်
- function များစွာအတွက် logic များ အသုံးချလိုသည်
- agent-level context ကို လက်လှမ်းထိလိုသည်
- လွှမ်းမိုးမှုများစွာ ရှိသောလုပ်ငန်းများ (log, auth စသည်)

❌ **middleware မသုံးသင့်သော အချိန်များ-**
- ရိုးရိုး input validation (Pydantic သုံးလိုက်ပါ)
- function-specific logic (function ထဲမှာသာထားပါ)
- တစ်ကြိမ်တည်း ပြောင်းလဲမှုလို (function ကိုသာ ပြောင်းပါ)

### 🎓 အဆင့်မြင့် ပုံစံများ-

```python
# အများကြီး middleware တွေ (အကောင်အထည်ဖော်စဉ် အစီအစဉ် အရေးကြီးပါတယ်!)
middleware=[
    logging_middleware,      # ပထမဆုံး မှတ်တမ်းတင်သည်
    auth_middleware,         # ထို့နောက် အာသာခံခြင်းစစ်ဆေးသည်
    cache_middleware,        # ထို့နောက် cache စစ်ဆေးသည်
    rate_limit_middleware,   # ထို့နောက် အမြန်နှုန်း ကန့်သတ်ချက်များ
    priority_check_middleware  # နောက်ဆုံးဦးစားပေးစစ်ဆေးမှု
]

# အခြေအနေမူတည်၍ middleware အကောင်အထည်ဖော်မှု
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # ရလဒ်ကို ပြင်ဆင်သည်
    else:
        # အကောင်အထည်ဖော်မှုကို လုံးဝ ကျော်လွှားသည်
        context.result = cached_value
```

### 🔗 ဆက်စပ် အကြောင်းအရာများ-

- **Agent Middleware**: agent.run() ခေါ်ဆိုမှုကို ထိတွေ့သည်
- **Function Middleware**: tool function ခေါ်ဆိုမှုကို ထိတွေ့သည် (ကျွန်တော်တို့ သုံးခဲ့သည်)
- **Middleware Pipeline**: middleware များတန်းစီဆောင်ရွက်မှု
- **Context Propagation**: middleware တန်းစီ ဖြတ်သန်း၍ state ပေးပို့ခြင်း


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**အတည်မပြုချက်**  
ဤစာတမ်းကို AI ဘာသာပြန်စနစ် [Co-op Translator](https://github.com/Azure/co-op-translator) ဖြင့် ဘာသာပြန်ထားပါသည်။ ပြည့်စုံမှုအတွက် ကြိုးစားထားပြီးဖြစ်သော်လည်း ကိုယ်ပိုင်အလိုအလျောက် ဘာသာပြန်ချက်များတွင် အမှားလျော့နည်းမှုများ ရှိနိုင်ကြောင်း သတိပြုပါရန် လိုအပ်သည်။ မူလစာတမ်းကို မူလဘာသာဖြင့်သာ တရားဝင် အရင်းအမြစ်အဖြစ် ယူဆရမည်ဖြစ်သည်။ အရေးကြီးသည့် အချက်အလက်များအတွက် အတည်ပြု လူ့ဘာသာပြန်နှင့် ပြန်ဆိုခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုအသုံးပြုမှုကနေ ဖြစ်ပေါ်လာသော အမှားနားလည်မှုများနှင့် တိရစ္ဆာန်မမှန်မှုများအတွက် ကျွန်ုပ်တို့ လွှဲအပ်မှုမခံပါ။
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
